In [1]:
import pulp
import numpy as np
import matplotlib_inline as plt

In [2]:
prob = pulp.LpProblem("Minimize_Cost", pulp.LpMinimize)
prob2 = pulp.LpProblem("Minimize_Cost", pulp.LpMinimize)
prob3 = pulp.LpProblem("Minimize_Cost", pulp.LpMinimize)

In [3]:
def generate_random_directed_graph(num_nodes, num_edges, cost_range=(1, 10)):
    # check the number of nodes and edges
    if num_edges < num_nodes or num_edges > num_nodes * (num_nodes - 1) // 2:
        raise ValueError("Number of edges must be at least num_nodes and no more than num_nodes * (num_nodes - 1) // 2.")

    # product nodes include the r
    nodes = ['r'] + [chr(i) for i in range(97, 97 + num_nodes - 1)]

    # create a cycle to ensure the strong connection
    cycle_edges = [(nodes[i], nodes[(i + 1) % num_nodes]) for i in range(num_nodes)]
    
    # produce other possible edge
    possible_edges = [(nodes[i], nodes[j]) for i in range(num_nodes) for j in range(num_nodes) if i != j and (nodes[j], nodes[i]) not in cycle_edges and (nodes[i], nodes[j]) not in cycle_edges]

    # remove the duplicated edge and self cycle
    selected_edges = cycle_edges
    remaining_edges_count = num_edges - len(cycle_edges)
    while remaining_edges_count > 0:
        np.random.shuffle(possible_edges)
        new_edge = possible_edges[0]
        if new_edge not in selected_edges and (new_edge[1], new_edge[0]) not in selected_edges:
            selected_edges = selected_edges + [new_edge]
            remaining_edges_count -= 1
    
    # produce edges with costs
    edges_with_costs = {edge: np.random.randint(cost_range[0], cost_range[1] + 1) for edge in selected_edges}

    return nodes, edges_with_costs

# 使用该函数生成一个随机有向图
num_nodes = 10# 节点数量，包括 'r'
num_edges = 45# 总边数，至少等于节点数
nodes, edges_with_costs = generate_random_directed_graph(num_nodes, num_edges)

print("Nodes:", nodes)
print("Directed edges with costs:", edges_with_costs)

Nodes: ['r', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']
Directed edges with costs: {('r', 'a'): 4, ('a', 'b'): 9, ('b', 'c'): 3, ('c', 'd'): 1, ('d', 'e'): 8, ('e', 'f'): 6, ('f', 'g'): 5, ('g', 'h'): 10, ('h', 'i'): 10, ('i', 'r'): 6, ('f', 'r'): 8, ('r', 'g'): 10, ('c', 'e'): 1, ('f', 'b'): 8, ('e', 'a'): 8, ('e', 'i'): 2, ('h', 'f'): 1, ('r', 'd'): 2, ('g', 'a'): 1, ('h', 'r'): 6, ('d', 'g'): 4, ('c', 'h'): 4, ('f', 'c'): 1, ('c', 'a'): 2, ('i', 'b'): 9, ('b', 'd'): 1, ('d', 'h'): 1, ('f', 'd'): 9, ('h', 'a'): 9, ('r', 'c'): 5, ('d', 'i'): 7, ('c', 'g'): 7, ('i', 'c'): 10, ('r', 'b'): 10, ('i', 'g'): 9, ('g', 'b'): 10, ('b', 'h'): 5, ('b', 'e'): 8, ('a', 'i'): 3, ('e', 'r'): 10, ('f', 'a'): 1, ('f', 'i'): 1, ('d', 'a'): 4, ('e', 'g'): 5, ('h', 'e'): 9}


In [4]:
#check no duplicated edge and self cycle
for (u, v) in edges_with_costs:
    if (v, u) in edges_with_costs:
        print([u,v])

In [0]:
'''
nodes = ['a','b','c','d','r']
edges_with_costs ={
    ('a','b'):3,('c','a'):1,('r','a'):9,('b','c'):8,
    ('b','r'):4,('d','b'):6,('r','c'):2,
    ('c','d'):7,('d','r'):5
}
'''

In [5]:
import pulp
x = pulp.LpVariable.dicts('x',(nodes,nodes),0,1,cat=pulp.LpInteger)
y = pulp.LpVariable.dicts('y',(nodes,nodes),0,1,cat=pulp.LpInteger)
z = pulp.LpVariable.dicts('z',(nodes,nodes),0,1,cat=pulp.LpInteger)

In [6]:
prob +=pulp.lpSum(edges_with_costs[u,v] * x[u][v] for u, v in edges_with_costs)
prob2 +=pulp.lpSum(edges_with_costs[u,v] * y[u][v] for u, v in edges_with_costs)
prob3 +=pulp.lpSum(edges_with_costs[u,v] * z[u][v] for u, v in edges_with_costs)

In [7]:
for v in nodes:
    if v != 'r':
        prob +=pulp.lpSum(x[u][v] for u in nodes if (u,v) in edges_with_costs) == 1
    prob3 +=pulp.lpSum(z[u][v] for u in nodes if (u,v) in edges_with_costs) >= 1

for u in nodes:
    if u != 'r':
        prob2 +=pulp.lpSum(y[u][v] for v in nodes if (u,v) in edges_with_costs) == 1
    prob3 +=pulp.lpSum(z[u][v] for v in nodes if (u,v) in edges_with_costs) >= 1

In [8]:
prob += pulp.lpSum(x[u]['r'] for u in nodes if (u,'r') in edges_with_costs) == 0
prob += pulp.lpSum(x['r'][v] for v in nodes if ('r',v) in edges_with_costs)>= 1

prob2 += pulp.lpSum(y[u]['r'] for u in nodes if (u,'r') in edges_with_costs) >=1
prob2 += pulp.lpSum(y['r'][v] for v in nodes if ('r',v) in edges_with_costs) ==0

In [9]:
from pulp import *
n = len(nodes)
subsets = [tuple(c) for c in allcombinations(nodes,n-1)]
subset = []
for s in subsets:
    if 'r' in s:
        subset.append(s)

In [10]:
for s in subset:
    sc = list(set(nodes)-set(s)) # complement of subset s
    summation = 0.0
    for u in s:
        for v in sc:
            if (u, v) in edges_with_costs: # Check if the (u, v) tuple is a valid key in edges_with_costs
                summation += x[u][v]
    prob += summation >= 1
    summation2 = 0.0
    for u in sc:
        for v in s:
            if (u, v) in edges_with_costs: # Check if the (u, v) tuple is a valid key in edges_with_costs
                summation2 += y[u][v]
    prob2 += summation2 >= 1
    
for s in subsets:
    sc = list(set(nodes) - set(s))  # complement of subset s
    summation3 = 0.0
    for u in s:
        for v in sc:
            if (u, v) in edges_with_costs:  # Check if the (u, v) tuple is a valid key in edges_with_costs
                summation3 += z[u][v]
    prob3 += summation3 >= 1  

In [11]:
prob.solve()
print ("Status:", LpStatus[prob.status])
prob2.solve()
print ("Status:", LpStatus[prob2.status])
prob3.solve()
print ("Status:", LpStatus[prob3.status])

Status: Optimal
Status: Optimal
Status: Optimal


In [12]:
print ("Optimal Solution for out")
resultout = []
for u in nodes:
	for v in nodes:
		if x[u][v].value() == 1:
			resultout.append((u,v))
resultout

Optimal Solution for out


[('r', 'd'),
 ('c', 'e'),
 ('d', 'g'),
 ('d', 'h'),
 ('f', 'a'),
 ('f', 'b'),
 ('f', 'c'),
 ('f', 'i'),
 ('h', 'f')]

In [13]:
print ("Optimal cost for out")
costout = 0.0
for e in resultout:
    costout+= edges_with_costs[e]
costout

Optimal cost for out


20.0

In [14]:
print ("Optimal Solution for in")
resultin = []
for u in nodes:
	for v in nodes:
		if y[u][v].value() == 1:
		    resultin.append((u,v))
resultin

Optimal Solution for in


[('a', 'i'),
 ('b', 'd'),
 ('c', 'e'),
 ('d', 'h'),
 ('e', 'i'),
 ('f', 'c'),
 ('g', 'a'),
 ('h', 'f'),
 ('i', 'r')]

In [15]:
print ("Optimal cost for in")
costin = 0.0
for e in resultin:
    costin+= edges_with_costs[e]
costin

Optimal cost for in


17.0

In [16]:
print ("Optimal Solution for strongly connected subgraph")
resultall = []
for u in nodes:
	for v in nodes:
		if z[u][v].value() == 1:
			resultall.append((u,v))
resultall

Optimal Solution for strongly connected subgraph


[('r', 'd'),
 ('a', 'b'),
 ('b', 'd'),
 ('c', 'e'),
 ('d', 'g'),
 ('d', 'h'),
 ('e', 'i'),
 ('f', 'c'),
 ('g', 'a'),
 ('h', 'f'),
 ('i', 'r')]

In [17]:
print ("Optimal Cost for strongly connected subgraph")
costall = 0.0
for e in resultall:
    costall+= edges_with_costs[e]
costall

Optimal Cost for strongly connected subgraph


29.0

In [18]:
print ("Combine solution for out and in")
result=[]
for u in nodes:
	for v in nodes:
		if x[u][v].value() == 1:
			result.append((u,v))
		elif y[u][v].value() == 1:
			result.append((u,v))
            
result

Combine solution for out and in


[('r', 'd'),
 ('a', 'i'),
 ('b', 'd'),
 ('c', 'e'),
 ('d', 'g'),
 ('d', 'h'),
 ('e', 'i'),
 ('f', 'a'),
 ('f', 'b'),
 ('f', 'c'),
 ('f', 'i'),
 ('g', 'a'),
 ('h', 'f'),
 ('i', 'r')]

In [19]:
print ("Combine Cost for out and in")
costcom = 0.0
for e in result:
    costcom+= edges_with_costs[e]
costcom

Combine Cost for out and in


33.0

In [20]:
print("Compare solution combine of out/in and for strongly connected subgraph")
print("result for strongly connected subgraph",resultall)
print("result for combine of out/in:", result)
print("Two result are same:", resultall==result)

Compare solution combine of out/in and for strongly connected subgraph
result for strongly connected subgraph [('r', 'd'), ('a', 'b'), ('b', 'd'), ('c', 'e'), ('d', 'g'), ('d', 'h'), ('e', 'i'), ('f', 'c'), ('g', 'a'), ('h', 'f'), ('i', 'r')]
result for combine of out/in: [('r', 'd'), ('a', 'i'), ('b', 'd'), ('c', 'e'), ('d', 'g'), ('d', 'h'), ('e', 'i'), ('f', 'a'), ('f', 'b'), ('f', 'c'), ('f', 'i'), ('g', 'a'), ('h', 'f'), ('i', 'r')]
Two result are same: False
